In [ ]:
!pip install aiohttp[speedups]
!pip install readability-lxml
!pip install trafilatura 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 263 kB 14.8 MB/s 
     |████████████████████████████████| 357 kB 67.9 MB/s 
     |████████████████████████████████| 291 kB 53.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 14.6 MB/s 
     |████████████████████████████████| 139 kB 82.3 MB/s 
     |████████████████████████████████| 837 kB 63.6 MB/s 
     |████████████████████████████████| 412 kB 67.1 MB/s 
     |████████████████████████████████| 288 kB 70.1 MB/s 
     |████████████████████████████████| 749 kB 72.9 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2
  Attempting uninstall: 

In [ ]:
!wget "https://docs.google.com/uc?export=download&id=1OodUg0U6-4yA8GLjrUeX4aeq-SLng_0J" -O "news_data_w_urls.tsv"
# !wget "https://docs.google.com/uc?export=download&id=1OodUg0U6-4yA8GLjrUeX4aeq-SLng_0J"

--2022-08-08 18:23:33--  https://docs.google.com/uc?export=download&id=1OodUg0U6-4yA8GLjrUeX4aeq-SLng_0J
Resolving docs.google.com (docs.google.com)... 108.177.127.113, 108.177.127.100, 108.177.127.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.127.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4u5i4vujmr69r5tbt025g0276f5j3k2o/1659982950000/07524398318767206543/*/1OodUg0U6-4yA8GLjrUeX4aeq-SLng_0J?e=download&uuid=506a22dc-9e5f-49e4-b0e6-2f19ac4fb04e [following]
--2022-08-08 18:23:34--  https://doc-0k-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4u5i4vujmr69r5tbt025g0276f5j3k2o/1659982950000/07524398318767206543/*/1OodUg0U6-4yA8GLjrUeX4aeq-SLng_0J?e=download&uuid=506a22dc-9e5f-49e4-b0e6-2f19ac4fb04e
Resolving doc-0k-08-docs.googleusercontent.com (doc-0k-08-docs.googleusercontent.com)... 108.177.127.132, 2a00:

In [ ]:
import pandas as pd
from readability import Document
import aiohttp
import requests
import trafilatura
import asyncio
import os
import time
import csv
import numpy as np
from IPython import display as ipydisplay

In [ ]:
data = pd.read_csv("news_data_w_urls.tsv", sep="\t")
data = data[np.invert(data["url"].str.contains("nytimes"))]  # paywalled
data = data[np.invert(data["url"].str.contains("wsj.com"))]  # paywalled
data = data[np.invert(data["url"].str.contains("reuters"))]  # all 404
data = data[np.invert(data["url"].str.contains("video."))]  # videos
data = data[np.invert(data["url"].str.contains("video/"))]  # videos

# data = data.sample(n=3500, replace=True)
# data.reset_index(inplace=True)

# data["text"] = None  # add new column

print(len(data))
data[:5]

16510


,url,q3,perceived,primary.topic,secondary.topic,democrat.vote,republican.vote
0,http://www.usatoday.com/story/news/nation/2013...,other,1,Civil Rights,Civil Rights,Neutral,Neutral
1,http://www.huffingtonpost.com/2013/05/21/senat...,News,0,Civil Rights,Civil Rights,Neutral,Neutral
2,http://www.washingtonpost.com/opinions/dont-le...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral
3,http://www.foxnews.com/politics/2013/04/03/oba...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral
4,http://www.breitbart.com/Big-Government/2013/1...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral


In [ ]:
categories = ["Negative", "SomewhatNegative", "Neutral", "SomewhatPositive", "Positive"]

dem_votes = [len(data[data["democrat.vote"] == vote]) for vote in categories]
rep_votes = [len(data[data["republican.vote"] == vote]) for vote in categories]

print(dem_votes, sum(dem_votes))
print(rep_votes, sum(rep_votes))

[988, 2714, 10698, 1784, 326] 16510
[928, 2503, 11802, 1098, 179] 16510


In [ ]:
mapping = {"1": [["Positive", "Negative"], ["Positive", "SomewhatNegative"], ["Positive", "Neutral"], ["Positive", "SomewhatPositive"],  # dem
                   ["SomewhatPositive", "Negative"], ["SomewhatPositive", "SomewhatNegative"], ["SomewhatPositive", "Neutral"]],
           
           "2": [["Negative", "Positive"], ["Negative", "Positive"], ["Neutral", "Positive"], ["SomewhatPositive", "Positive"],  # rep
                   ["Negative", "SomewhatPositive"], ["SomewhatNegative", "SomewhatPositive"], ["Neutral", "SomewhatPositive"]],
           
           "3": [["Neutral", "Neutral"], ["SomewhatPositive", "SomewhatPositive"], ["Positive", "Positive"]],  # neutral
           
           False: [["Negative", "Negative"], ["SomewhatNegative", "SomewhatNegative"], ["SomewhatNegative",	"Neutral"], ["Neutral", "SomewhatNegative"], ["Negative",	"Neutral"], ["Neutral", "Negative"]]}

data["label"] = None
for i, row in data.iterrows():
  for k, v in mapping.items():
    if [data.loc[i]["democrat.vote"], data.loc[i]["republican.vote"]] in v:
      data.loc[i, "label"] = k
 
display(data[:20])

,url,q3,perceived,primary.topic,secondary.topic,democrat.vote,republican.vote,label
0,http://www.usatoday.com/story/news/nation/2013...,other,1,Civil Rights,Civil Rights,Neutral,Neutral,3
1,http://www.huffingtonpost.com/2013/05/21/senat...,News,0,Civil Rights,Civil Rights,Neutral,Neutral,3
2,http://www.washingtonpost.com/opinions/dont-le...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,3
3,http://www.foxnews.com/politics/2013/04/03/oba...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,3
4,http://www.breitbart.com/Big-Government/2013/1...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,3
5,http://www.cnn.com/2013/06/28/politics/obama-c...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,3
6,http://www.foxnews.com/politics/2013/10/14/ami...,News,0,Civil Rights,Civil Rights,Neutral,SomewhatPositive,2
7,http://www.bbc.co.uk/news/technology-22213379,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,3
8,http://www.nbcnews.com/technology/microsoft-le...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,3
9,http://radio.foxnews.com/toddstarnes/top-stori...,other,0,Civil Rights,Civil Rights,Neutral,Neutral,3


In [ ]:
print(len(data[data["label"] == "1"]))
print(len(data[data["label"] == "2"]))
print(len(data[data["label"] == "3"]))

size = 1050

dem_labelled = data[data["label"] == "1"].sample(n=size)
rep_labelled = data[data["label"] == "2"].sample(n=size)
neu_labelled = data[data["label"] == "3"].sample(n=size)

data = pd.concat([dem_labelled, rep_labelled, neu_labelled])
data.reset_index(inplace=True, drop=True)

NameError: ignored

In [ ]:
len(data)

3150

In [ ]:
session = aiohttp.ClientSession()  # prob bad practice but im lazy and tired
concurrent_download_limiter = asyncio.Semaphore(16)

async def async_get_article_text(url):
    async with concurrent_download_limiter:
        async with session.get(url) as response:
                print(f"FETCHING: {url}")
                try:
                    text = trafilatura.extract(await response.text())
                except exception as e:
                    print(f"{e} while FETCHING: {url}")
                    text = None
                return text

async def update_row_article_text(row_index, url):
    text = await async_get_article_text(url)
    data.loc[row_index, "text"] = text
    print(f"{row_index}/{len(data)}")
    ipydisplay.clear_output(wait=True)
    
    
loop = asyncio.get_event_loop()
async def main():
    for row in data.iterrows():
        loop.create_task(update_row_article_text(row[0], row[1]["url"]))
        await asyncio.sleep(0.35) # hopefully i dont get banned or throttled
    data.to_csv("news_data_w_text.csv")
loop.create_task(main())

<Task pending coro=<main() running at <ipython-input-42-8bb68cebc29f>:23>>

In [ ]:
data[:-40]

,url,q3,perceived,primary.topic,secondary.topic,democrat.vote,republican.vote,label,text
2284,http://news.yahoo.com/john-kerry-throws-freedo...,Opinion,0,Civil Rights,Elections,SomewhatPositive,Negative,1,NaN
14477,http://www.huffingtonpost.com/2013/10/07/obama...,News,0,Healthcare,None,SomewhatPositive,Neutral,1,By Lewis Krauskopf\nOct 7 (Reuters) - The Obam...
8031,http://www.latimes.com/nation/la-na-congress-b...,News,0,Economy,None,SomewhatPositive,Neutral,1,Budget deal faces test in House\nCongressional...
12552,http://news.yahoo.com/blogs/lookout/obama-end-...,News,1,Gun Related Crimes,None,SomewhatPositive,Neutral,1,Can Obama end the long fight over gun violence...
11858,http://www.dailykos.com/story/2013/01/03/11759...,Opinion,0,Elections,None,SomewhatPositive,Negative,1,Access Denied! (403)\nYou do not have permissi...
...,...,...,...,...,...,...,...,...,...
6350,http://www.chicagotribune.com/news/local/ct-me...,News,1,National Security,National Security,Neutral,Neutral,3,NaN
16621,http://www.huffingtonpost.com/2013/03/01/dick-...,News,0,National Security,None,Neutral,Neutral,3,NaN
12674,http://www.chicagotribune.com/news/local/ct-me...,News,1,Gun Related Crimes,None,Neutral,Neutral,3,NaN
13825,http://www.chicagotribune.com/business/breakin...,News,0,Healthcare,None,Neutral,Neutral,3,NaN


FETCHING: http://www.latimes.com/business/la-fi-obama-cyber-20130221,0,5653214.story
5050/3150


In [ ]:
data = data[data["text"] != None]
data = data[data["text"] != ""]
data = data[np.invert(data["text"].str.lower().str.contains("page not found").astype(bool))]
data = data[np.invert(data["text"].str.lower().str.contains("40").astype(bool))]
data = data[np.invert(data["text"].str.lower().str.contains("something has gone wrong…\nit seems you clicke").astype(bool))]

data.to_csv("news_data_w_text.csv")


## Alternative (but i think overall slightly less effective) method - performs better in some aspects so kept for reference

In [ ]:
# text_extractor = html2text.HTML2Text()
# text_extractor.ignore_links = True
# text_extractor.ignore_images = True
# text_extractor.ignore_tables = True
# text_extractor.ignore_ephasis = True

In [ ]:
# response = requests.get('http://www.cnn.com/2013/06/28/politics/obama-contraceptives/index.html')
# decluttered = Document(response.text)  # removes headers, footers, etc.
# text = text_extractor.handle(decluttered.summary())  # grabs text (doc output is still html)
# print(text)
# print(len(text))



Washington CNN —

The Obama administration finalized rules on Friday that allow religious-
affiliated organizations opposing the use of contraception to opt out of a
federal mandate requiring that they provide their employees with insurance
coverage for birth control.

The mandates give women at nonprofit, religious-based organizations, like
certain hospitals and universities, the ability to receive contraception
through separate health policies at no charge.

The rules, which were first proposed in February and then open for comment
through April, have undergone only minor changes. Chiquita Brooks-LaSure,
deputy director for policy and regulations at the U.S. Department of Health
and Human Services, said in a call with reporters that the rules were “very
similar” to the administration’s original proposal.

One of the few noticeable changes is the process by which insurance companies
reimburse nonprofit religious organizations – such as nonprofit religious
hospitals and institutions 